In [1]:
import numpy as np
from numba import njit
import pickle
from sklearn.ensemble import BaggingClassifier
from sklearn.tree._classes import DecisionTreeClassifier
from joblib import Parallel, delayed
from scipy.stats import entropy, multivariate_normal

from hyppo.tools import multimodal_independence, indep_sim
from hyppo.ksample._utils import k_sample_transform
from tqdm import tqdm

import sys
import os
import multiprocessing as mp
from joblib import Parallel, delayed

C:\Users\siptest\anaconda3\envs\py36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\siptest\anaconda3\envs\py36\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\siptest\anaconda3\envs\py36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\siptest\anaconda3\envs\py36\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\Users\siptest\anaconda3\envs\py36\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [5]:
def test_stat_helper(est_nodes, eval_nodes, est_classes, class_counts, K, kappa=3, base=2):
    """
    est_nodes : list
        Leaf indices used for voting
    eval_nodes : list
        Leaf indices in which a sample from the density subsample falls
    est_classes : list
        Voter class labels in est_node leaves
    """
    for i in range(len(est_nodes)):
        class_counts[est_nodes[i], est_classes[i]] += 1

    # Total number of estimation points in each leaf.
    row_sums = class_counts.sum(axis=1)
    row_sums[row_sums == 0] = 1  # Avoid divide by zero.
    class_probs = class_counts / row_sums[:, None]

    # Make the nodes that have no estimation indices uniform.
    # This includes non-leaf nodes, but tha t will not affect the estimate.
    class_probs[np.argwhere(class_probs.sum(axis=1) == 0)] = [1 / K]*K
    # Apply finite sample correction and renormalize.
    where_0 = np.argwhere(class_probs == 0)
    for elem in where_0:
        class_probs[elem[0], elem[1]] = 1 / \
            (kappa*class_counts.sum(axis=1)[elem[0]])
    row_sums = class_probs.sum(axis=1)
    class_probs = class_probs / row_sums[:, None]

    # Place evaluation points in their corresponding leaf node.
    # Store evaluation posterior in a num_eval-by-num_class matrix.
    eval_class_probs = class_probs[eval_nodes]
    eval_entropies = [entropy(posterior, base=base)
                      for posterior in eval_class_probs]
    return np.mean(eval_entropies)


In [3]:
def uf(X, y, n_estimators=300, max_samples=.4, base=2, kappa=3, reps=100, n_jobs=None):
    # Build forest with default parameters.
    model = BaggingClassifier(DecisionTreeClassifier(),
                              n_estimators=n_estimators,
                              max_samples=max_samples,
                              n_jobs=n_jobs,
                              bootstrap=False)
    model.fit(X, y)
    n = X.shape[0]
    K = model.n_classes_
    _, y = np.unique(y, return_inverse=True)

    cond_entropy = 0
    final_null_dist = [0] * 100

    tree_est_nodes = []
    tree_eval_nodes = []
    tree_unsampled_indices = []

    # Get real test statistics
    for tree_idx, tree in enumerate(model):
        # Find the indices of the training set used for partition.
        sampled_indices = model.estimators_samples_[tree_idx]
        unsampled_indices = np.delete(np.arange(0, n), sampled_indices)
        np.random.shuffle(unsampled_indices)
        tree_unsampled_indices.append(unsampled_indices)

        # Randomly split the rest into voting and evaluation.
        vote_indices = unsampled_indices[:len(unsampled_indices)//2]
        eval_indices = unsampled_indices[len(unsampled_indices)//2:]

        # Store the posterior in a num_nodes-by-num_classes matrix.
        # Posteriors in non-leaf cells will be zero everywhere
        # and later changed to uniform.
        node_counts = tree.tree_.n_node_samples
        class_counts = np.zeros((len(node_counts), K))
        est_nodes = tree.apply(X[vote_indices])
        tree_est_nodes.append(est_nodes)
        eval_nodes = tree.apply(X[eval_indices])
        tree_eval_nodes.append(eval_nodes)

        cond_entropy += test_stat_helper(
            est_nodes, eval_nodes, y[vote_indices], class_counts, K)

    # Generate null dist
    for j in range(reps):
        for tree, unsampled_indices, est_nodes, eval_nodes in zip(
            model, tree_unsampled_indices, tree_est_nodes, tree_eval_nodes
        ):
            node_counts = tree.tree_.n_node_samples
            class_counts = np.zeros((len(node_counts), K))
            y_vote = y[unsampled_indices]
            np.random.shuffle(y_vote)
            final_null_dist[j] += test_stat_helper(
                est_nodes, eval_nodes, y_vote[:len(unsampled_indices)//2], class_counts, K)

    # note: shuffling y doesn't change these outputs
    new_final_null_dist = [entropy([np.mean(
        y), 1 - np.mean(y)], base=2) - val / n_estimators for val in final_null_dist]

    final_stat = entropy([np.mean(y), 1 - np.mean(y)],
                         base=2) - cond_entropy / n_estimators
    return final_stat, new_final_null_dist

In [16]:
MAX_SAMPLE_SIZE = 500
STEP_SIZE = 100 
SAMP_SIZES = range(100, MAX_SAMPLE_SIZE + STEP_SIZE, STEP_SIZE)
POWER_REPS = 100

SIMULATIONS = [
    # "linear": "Linear",
    # "multimodal_independence": "Independence"
    # linear,
    multimodal_independence
]

In [17]:
from statsmodels.distributions.empirical_distribution import ECDF
def estimate_power(sim, n_jobs=None):
    samp_size_dict = dict()
    samp_size_dict['sample_sizes'] = SAMP_SIZES
    samp_size_dict['n_power_reps'] = POWER_REPS
    power = []
    for n_samples in SAMP_SIZES:
        pvalues = []
        samp_size_dict[n_samples] = {'stats': [], 'null_dists': []}
        for p in tqdm(range(POWER_REPS)):
            #plt.clf()
            np.random.seed(None)
            matrix1, matrix2 = multimodal_independence(n_samples, 2)
            x, y = k_sample_transform([matrix1, matrix2])
            stat, null_dist = uf(x, y.ravel(), n_jobs=n_jobs)
            samp_size_dict[n_samples]['stats'].append(stat)
            samp_size_dict[n_samples]['null_dists'].append(null_dist)
            pvalue = np.mean(np.asarray(null_dist) >= stat)
            print("P-value: " + str(pvalue))
            print(f'Test stat: {stat}')
            print(f'Null dist: {null_dist[:5]}')
            #if p % 4 == 0: 
                #plt.hist(null_dist)
                #plt.axvline(stat, c='r', ls='--')
            #plt.show()
            pvalues.append(pvalue)
        #plt.hist(pvalues, range=(0,1))
        #plt.show()
        ecdf = ECDF(pvalues)
        plt.plot(ecdf.x, ecdf.y)
        plt.gca().set_aspect('equal', adjustable='box')
        plt.xlabel("p-value")
        plt.xlim(0, 1)
        plt.ylim(0, 1)
        plt.ylabel("fraction of data")
        plt.show()
        power.append(np.mean(np.asarray(pvalues) <= 0.05))
        #power.append((pvalues >= 0.05).sum() / POWER_REPS)

    with open('multimodal_independence_power_reps.pkl', 'wb') as handle:
        pickle.dump(samp_size_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

    np.savetxt('C:/Users/siptest/Desktop/NDD/multimodal_independence_power1000.csv',
               power, delimiter=',')

    return power

In [ ]:
estimate_power(SIMULATIONS[0], n_jobs=4)

  1%|▊                                                                               | 1/100 [01:07<1:51:32, 67.61s/it]

P-value: 0.0
Test stat: 0.14795680164144165
Null dist: [0.14186699356589638, 0.14405184297227547, 0.1398056151681789, 0.13841311445315574, 0.14274966964198899]


  2%|█▌                                                                              | 2/100 [02:15<1:50:34, 67.70s/it]

P-value: 0.0
Test stat: 0.14406575287598244
Null dist: [0.14247201716999391, 0.1375670084472832, 0.13883579934323165, 0.13705964486178768, 0.14043367597632372]


  3%|██▍                                                                             | 3/100 [03:23<1:49:21, 67.64s/it]

...

P-value: 0.98
Test stat: 0.1344977393783402
Null dist: [0.14057070630304924, 0.1419255521436531, 0.13988124047211614, 0.13704933959811816, 0.13919282390565246]


 18%|██████████████▏                                                                | 18/100 [21:53<1:43:22, 75.64s/it]

P-value: 0.14
Test stat: 0.14139274774201305
Null dist: [0.14134306187018186, 0.13945399882442933, 0.14051620227611994, 0.13930057036519583, 0.13840825101069276]


 19%|███████████████                                                                | 19/100 [23:06<1:40:57, 74.79s/it]

P-value: 0.01
Test stat: 0.1418131959216199
Null dist: [0.13454651803143203, 0.13588355420049847, 0.13521972676199956, 0.1374340097087463, 0.1377524425853488]


In [12]:
import matplotlib.pyplot as plt
def plot_power2(): 
    
    sim_title = [
        #"Linear", 
        "Independence"
    ]
    sim = multimodal_independence
    power = np.genfromtxt('C:/Users/siptest/Desktop/NDD/multimodal_independence_power1000.csv',
                                      delimiter=',')
    plt.plot(power)
    plt.yticks([0, 1])
    plt.axhline(y=0.05, color='b', linestyle='--')
    positions = (0, 3, 5)
    labels = ("100", "65", "105")
    plt.xticks(positions, labels)
    plt.xlabel("Sample Size")
    plt.ylabel("Mean Power from 100 Reps")
    plt.savefig("C:/Users/siptest/Desktop/NDD/Independence_UF_HonestSampling1000.jpg", bbox_inches='tight')
    

In [13]:
plot_power2()

OSError: C:/Users/siptest/Desktop/NDD/multimodal_independence_power1000.csv not found.